In [1]:
from scrapping import *
from fonctions_preprocess import *
pd.set_option('display.max_columns', None) 

In [30]:
from fonctions_preprocess import *
from scrapping import *


def update_data_global():

    """
    Met à jour les données de football en récupérant les dernières données scrapeées,
    en traitant ces données, et en les fusionnant avec les données existantes.
    """

    mapping_equipe = {
    'Nimes': 'Nîmes',
    'Paris S-G': 'Paris Saint Germain',
    'Saint Etienne': 'Saint-Étienne'
    }

    # Obtenir la date et l'heure actuelles
    current_datetime = datetime.now()

    # Lire le fichier CSV contenant les résultats futurs
    future_results = pd.read_csv("/Users/alexandre/Downloads/python-ensae/Projet-python/Fbref_alex/results.csv", index_col=0)
    future_results["DateTime"] = pd.to_datetime(future_results["DateTime"])

    # Vérifier si la dernière date dans le fichier est antérieure à la date actuelle
    last_date_in_file = future_results["DateTime"].max()
    if last_date_in_file <= current_datetime:

        # Lire le fichier CSV de la base de données actuelle
        current_database = pd.read_csv("/Users/alexandre/Downloads/python-ensae/Projet-python/Fbref_alex/dynamic_soccer_database.csv", index_col=0)

        # Scrapper les dernières données de Ligue 1
        latest_data = scrape_latest_ligue1_data()
        new_data = latest_data[0].copy()

        # Ajouter les nouveaux matchs à la base de données actuelle
        updated_database = add_new_matches(base_initiale=current_database, base_nouvelle=new_data)
        updated_database.to_csv("/Users/alexandre/Downloads/python-ensae/Projet-python/Fbref_alex/dynamic_soccer_database.csv")

        # Préparer les données pour les prochaines semaines de match
        upcoming_matchweeks = latest_data[1].copy()
        upcoming_matchweeks = find_futur_matchweeks(upcoming_matchweeks, mapping_equipe)

        # Traiter le cas où il n'y a pas de nouveaux matchs à récupérer
        if upcoming_matchweeks is None:
            return
        else:
            # Traiter les données des prochaines semaines de match
            processed_upcoming_matches = preprocess_initial(upcoming_matchweeks.copy(), mapping_equipe)
            processed_upcoming_matches = renommer_colonnes(processed_upcoming_matches)
            processed_upcoming_matches = columns_to_keep(processed_upcoming_matches)

            # Traiter les données mises à jour
            processed_updated_database = preprocess_initial(updated_database.copy(), mapping_equipe)
            processed_updated_database = renommer_colonnes(processed_updated_database)
            processed_updated_database = columns_to_keep(processed_updated_database)
            processed_updated_database = preprocess_variables(processed_updated_database)

            # Fusionner les données traitées
            combined_data = pd.concat([processed_updated_database, processed_upcoming_matches], sort=False).reset_index(drop=True)
            combined_data = preparation_model(combined_data)
            combined_data = preprocess_data(combined_data)

            # Mettre à jour la colonne 'Result' dans future_results
            future_results = future_results.merge(combined_data[['MatchID', 'Result']], on='MatchID', how='left', suffixes=('', '_from_combined'))
            future_results['Result'].update(future_results['Result_from_combined'])
            future_results.drop(columns=['Result_from_combined'], inplace=True)

            # Traiter les données pour la modélisation et la mise à jour finale
            final_result = modelisation(combined_data, current_datetime)
            future_results = pd.concat([future_results, final_result]).sort_values(by="DateTime", ascending=False).reset_index(drop=True)

            # Enregistrer les résultats futurs mis à jour
            future_results.to_csv("Fbref_alex/results.csv")

    return

#update_data_global()

In [36]:
B = pd.read_csv("/Users/alexandre/Downloads/python-ensae/Projet-python/Fbref_alex/dynamic_soccer_database.csv", index_col=0)
#C = pd.read_csv("/home/onyxia/work/Projet-python/Fbref_alex/recup_matchweek.csv", index_col=0)
mapping_equipe = {
'Nimes': 'Nîmes',
'Paris S-G': 'Paris Saint Germain',
'Saint Etienne': 'Saint-Étienne'
}

In [7]:
C = scrape_latest_ligue1_data()

In [37]:
Y = C[1].copy()
Y = find_futur_matchweeks(Y, mapping_equipe)


In [38]:
B = preprocess_initial(B, mapping_equipe)
Y = preprocess_initial(Y, mapping_equipe)
B = renommer_colonnes(B)
Y = renommer_colonnes(Y)
B = columns_to_keep(B)
Y = columns_to_keep(Y)
B = preprocess_variables(B)
combined_data = pd.concat([Y, B], sort=False).reset_index(drop=True)

B = preparation_model(combined_data)
B = preprocess_data(combined_data)




/Users/alexandre/Downloads/python-ensae/Projet-python/Fbref_alex/fonctions_preprocess.py:169: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df['FormeW_Lag'] = df.groupby(['Saison', 'Team'])['IsWin'].apply(lambda x: x.shift(1).rolling(window=5, min_periods=5).sum())
/Users/alexandre/Downloads/python-ensae/Projet-python/Fbref_alex/fonctions_preprocess.py:170: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_ke

In [27]:
Y.head(1)

,DateTime,Comp,Round,Day,Venue,Result,GF,GA,Opponent,MatchID,Saison,Attendance,Captain,Formation,Referee,Match Report,Notes,Team
0,2023-08-12 21:00:00,Ligue 1,1,Sat,Home,D,0.0,0.0,Lorient,Paris Saint Germain_Lorient,2023-2024,47000.0,Danilo Pereira,4-3-3,Bastien Dechepy,Match Report,NaN,Paris Saint Germain


In [68]:
F = modelisation(B, datetime(2023, 12, 20))


In [69]:
RRRR = F[F["Round"]==17]

RRRR["GF_Home"] = np.nan

RRRR["GF_Away"] = np.nan

RRRR["Result"] = np.nan

RRRR.to_csv("/Users/alexandre/Downloads/python-ensae/Projet-python/Fbref_alex/results.csv")


/var/folders/tm/y39_tg8n68d9hjl53sj946g00000gn/T/ipykernel_52605/3969815986.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RRRR["GF_Home"] = np.nan
/var/folders/tm/y39_tg8n68d9hjl53sj946g00000gn/T/ipykernel_52605/3969815986.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RRRR["GF_Away"] = np.nan
/var/folders/tm/y39_tg8n68d9hjl53sj946g00000gn/T/ipykernel_52605/3969815986.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

In [70]:
from fonctions_preprocess import *
from scrapping import *


def update_data_global():

    """
    Met à jour les données de football en récupérant les dernières données scrapeées,
    en traitant ces données, et en les fusionnant avec les données existantes.
    """

    mapping_equipe = {
    'Nimes': 'Nîmes',
    'Paris S-G': 'Paris Saint Germain',
    'Saint Etienne': 'Saint-Étienne'
    }

    # Obtenir la date et l'heure actuelles
    current_datetime = datetime.now()

    # Lire le fichier CSV contenant les résultats futurs
    future_results = pd.read_csv("/Users/alexandre/Downloads/python-ensae/Projet-python/Fbref_alex/results.csv", index_col=0)
    future_results["DateTime"] = pd.to_datetime(future_results["DateTime"])

    # Vérifier si la dernière date dans le fichier est antérieure à la date actuelle
    last_date_in_file = future_results["DateTime"].max()
    if last_date_in_file <= current_datetime:

        # Lire le fichier CSV de la base de données actuelle
        current_database = pd.read_csv("/Users/alexandre/Downloads/python-ensae/Projet-python/Fbref_alex/dynamic_soccer_database.csv", index_col=0)

        # Scrapper les dernières données de Ligue 1
        latest_data = scrape_latest_ligue1_data()
        new_data = latest_data[0].copy()

        # Ajouter les nouveaux matchs à la base de données actuelle
        updated_database = add_new_matches(base_initiale=current_database, base_nouvelle=new_data)
        updated_database.to_csv("/Users/alexandre/Downloads/python-ensae/Projet-python/Fbref_alex/dynamic_soccer_database.csv")

        # Préparer les données pour les prochaines semaines de match
        upcoming_matchweeks = latest_data[1].copy()
        upcoming_matchweeks = find_futur_matchweeks(upcoming_matchweeks, mapping_equipe)

        # Traiter le cas où il n'y a pas de nouveaux matchs à récupérer
        if upcoming_matchweeks is None:
            return
        else:
            # Traiter les données des prochaines semaines de match
            processed_upcoming_matches = preprocess_initial(upcoming_matchweeks.copy(), mapping_equipe)
            processed_upcoming_matches = renommer_colonnes(processed_upcoming_matches)
            processed_upcoming_matches = columns_to_keep(processed_upcoming_matches)

            # Traiter les données mises à jour
            processed_updated_database = preprocess_initial(updated_database.copy(), mapping_equipe)
            processed_updated_database = renommer_colonnes(processed_updated_database)
            processed_updated_database = columns_to_keep(processed_updated_database)
            processed_updated_database = preprocess_variables(processed_updated_database)

            # Fusionner les données traitées
            combined_data = pd.concat([processed_updated_database, processed_upcoming_matches], sort=False).reset_index(drop=True)
            combined_data = preparation_model(combined_data)
            combined_data = preprocess_data(combined_data)

            """
            # Mettre à jour la colonne 'Result' dans future_results
            future_results = future_results.merge(combined_data[['MatchID', 'Result']], on='MatchID', how='left', suffixes=('', '_from_combined'))
            future_results['Result'].update(future_results['Result_from_combined'])
            future_results.drop(columns=['Result_from_combined'], inplace=True)
            """
            # Mettre à jour la colonne 'Result'
            future_results = future_results.merge(combined_data[['MatchID', 'Result']], on='MatchID', how='left', suffixes=('', '_from_combined'))
            future_results['Result'].update(future_results['Result_from_combined'])
            future_results.drop(columns=['Result_from_combined'], inplace=True)

            # Mettre à jour la colonne 'Minus 2.5 Goals', si cette colonne existe dans combined_data
            if 'GF_Home' in combined_data.columns:
                future_results = future_results.merge(combined_data[['MatchID', 'GF_Home']], on='MatchID', how='left', suffixes=('', '_from_combined'))
                future_results['GF_Home'].update(future_results['GF_Home_from_combined'])
                future_results.drop(columns=['GF_Home_from_combined'], inplace=True)

            # Mettre à jour la colonne 'Minus 2.5 Goals', si cette colonne existe dans combined_data
            if 'GF_Away' in combined_data.columns:
                future_results = future_results.merge(combined_data[['MatchID', 'GF_Away']], on='MatchID', how='left', suffixes=('', '_from_combined'))
                future_results['GF_Away'].update(future_results['GF_Away_from_combined'])
                future_results.drop(columns=['GF_Away_from_combined'], inplace=True)

            # Traiter les données pour la modélisation et la mise à jour finale
            final_result = modelisation(combined_data, current_datetime)
            future_results = pd.concat([future_results, final_result]).sort_values(by="DateTime", ascending=False).reset_index(drop=True)

            # Enregistrer les résultats futurs mis à jour
            future_results.to_csv("/Users/alexandre/Downloads/python-ensae/Projet-python/Fbref_alex/results.csv")
    return

In [71]:
update_data_global()

/Users/alexandre/Downloads/python-ensae/Projet-python/Fbref_alex/fonctions_preprocess.py:169: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df['FormeW_Lag'] = df.groupby(['Saison', 'Team'])['IsWin'].apply(lambda x: x.shift(1).rolling(window=5, min_periods=5).sum())
/Users/alexandre/Downloads/python-ensae/Projet-python/Fbref_alex/fonctions_preprocess.py:170: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_ke

In [72]:
pd.read_csv("results.csv", index_col=0)

,DateTime,Comp,Saison,Round,Day,Team Home,GF_Home,GF_Away,Team Away,Result,Predicted_Result,Prediction_Probability_Result,Predicted_Minus 2.5 Goals,Prediction_Probability_Minus 2.5 Goals,MatchID
0,2024-01-14 20:45:00,Ligue 1,2023-2024,18,Sun,Lens,NaN,NaN,Paris Saint Germain,NaN,D,0.41,1.0,0.57,2024-01-14 20:45:00-Lens-vs-Paris Saint Germain
1,2024-01-14 17:05:00,Ligue 1,2023-2024,18,Sun,Le Havre,NaN,NaN,Lyon,NaN,W_Away,0.52,1.0,0.61,2024-01-14 17:05:00-Le Havre-vs-Lyon
2,2024-01-14 15:00:00,Ligue 1,2023-2024,18,Sun,Brest,NaN,NaN,Montpellier,NaN,W_Home,0.45,0.0,0.51,2024-01-14 15:00:00-Brest-vs-Montpellier
3,2024-01-14 15:00:00,Ligue 1,2023-2024,18,Sun,Nantes,NaN,NaN,Clermont Foot,NaN,W_Home,0.37,1.0,0.52,2024-01-14 15:00:00-Clermont Foot-vs-Nantes
4,2024-01-14 15:00:00,Ligue 1,2023-2024,18,Sun,Metz,NaN,NaN,Toulouse,NaN,W_Away,0.55,1.0,0.64,2024-01-14 15:00:00-Metz-vs-Toulouse
5,2024-01-14 13:00:00,Ligue 1,2023-2024,18,Sun,Lille,NaN,NaN,Lorient,NaN,W_Home,0.59,1.0,0.57,2024-01-14 13:00:00-Lille-vs-Lorient
6,2024-01-13 21:00:00,Ligue 1,2023-2024,18,Sat,Rennes,NaN,NaN,Nice,NaN,W_Away,0.40,1.0,0.56,2024-01-13 21:00:00-Nice-vs-Rennes
7,2024-01-13 17:00:00,Ligue 1,2023-2024,18,Sat,Monaco,NaN,NaN,Reims,NaN,W_Home,0.63,0.0,0.50,2024-01-13 17:00:00-Monaco-vs-Reims
8,2024-01-12 21:00:00,Ligue 1,2023-2024,18,Fri,Marseille,NaN,NaN,Strasbourg,NaN,W_Home,0.45,1.0,0.52,2024-01-12 21:00:00-Marseille-vs-Strasbourg
9,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Clermont Foot,1.0,3.0,Rennes,W_Away,D,0.40,1.0,0.75,2023-12-20 21:00:00-Clermont Foot-vs-Rennes


In [9]:
a = pd.read_csv("/Users/alexandre/Downloads/python-ensae/Projet-python/Fbref_alex/results.csv", index_col=0)
a[a["Round"]==17]

,DateTime,Comp,Saison,Round,Day,Team Home,Team Away,Result,Predicted_Result,Prediction_Probability,MatchID
9,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Toulouse,Monaco,W_Away,W_Away,0.55,2023-12-20 21:00:00-Monaco-vs-Toulouse
10,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Clermont Foot,Rennes,W_Away,W_Away,0.47,2023-12-20 21:00:00-Clermont Foot-vs-Rennes
11,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Lyon,Nantes,W_Home,W_Home,0.58,2023-12-20 21:00:00-Lyon-vs-Nantes
12,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Montpellier,Marseille,D,D,0.44,2023-12-20 21:00:00-Marseille-vs-Montpellier
13,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Strasbourg,Lille,W_Home,D,0.42,2023-12-20 21:00:00-Lille-vs-Strasbourg
14,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Reims,Le Havre,W_Home,D,0.44,2023-12-20 21:00:00-Le Havre-vs-Reims
15,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Brest,Lorient,W_Home,W_Home,0.49,2023-12-20 21:00:00-Brest-vs-Lorient
16,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Nice,Lens,W_Home,W_Home,0.45,2023-12-20 21:00:00-Lens-vs-Nice
17,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Paris Saint Germain,Metz,W_Home,W_Home,0.72,2023-12-20 21:00:00-Metz-vs-Paris Saint Germain


In [5]:
mapping_equipe = {
    'Nimes': 'Nîmes',
    'Paris S-G': 'Paris Saint Germain',
    'Saint Etienne': 'Saint-Étienne'
    }
pd.set_option('display.max_columns', None)

In [6]:


ajd = datetime.now()

B = preprocess_initial(B, mapping_equipe=mapping_equipe)
C = preprocess_initial(C, mapping_equipe)
B = renommer_colonnes(B)
C = renommer_colonnes(C)
B = columns_to_keep(B)
C = columns_to_keep(C)
B = preprocess_variables(B)
resultat = pd.concat([C, B], sort=False).reset_index(drop= True)
resultat = preparation_model(B)
resultat = preprocess_data(B)


KeyError: 'Opponent'

In [29]:
a=modelisation(resultat, datetime(2023, 12, 16, 21, 0, 0))
a["Result"]=np.NaN




In [30]:
# Fusionner les DataFrames
a = a.merge(resultat[['MatchID', 'Result']], on='MatchID', how='left', suffixes=('', '_from_resultat'))

# Mettre à jour la colonne 'Result' dans a
a['Result'].update(a['Result_from_resultat'])

# Supprimer la colonne supplémentaire
a.drop(columns=['Result_from_resultat'], inplace=True)


/tmp/ipykernel_288800/2344193416.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['D' 'W_Home' 'W_Away' 'W_Away' 'D' 'W_Away' 'W_Home' 'W_Home' 'W_Home'
 'W_Home' 'W_Home' 'D' 'W_Home' 'W_Away' 'W_Away']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  a['Result'].update(a['Result_from_resultat'])


In [31]:
a

,DateTime,Comp,Saison,Round,Day,Team Home,Team Away,Result,Predicted_Result,Prediction_Probability,MatchID
0,2023-12-17 20:45:00,Ligue 1,2023-2024,16,Sun,Lille,Paris Saint Germain,D,W_Away,0.53,2023-12-17 20:45:00-Lille-vs-Paris Saint Germain
1,2023-12-17 17:05:00,Ligue 1,2023-2024,16,Sun,Marseille,Clermont Foot,W_Home,W_Home,0.44,2023-12-17 17:05:00-Clermont Foot-vs-Marseille
2,2023-12-17 13:00:00,Ligue 1,2023-2024,16,Sun,Nantes,Brest,W_Away,W_Away,0.40,2023-12-17 13:00:00-Brest-vs-Nantes
3,2023-12-17 15:00:00,Ligue 1,2023-2024,16,Sun,Metz,Montpellier,W_Away,W_Home,0.41,2023-12-17 15:00:00-Metz-vs-Montpellier
4,2023-12-17 15:00:00,Ligue 1,2023-2024,16,Sun,Toulouse,Rennes,D,W_Away,0.46,2023-12-17 15:00:00-Rennes-vs-Toulouse
5,2023-12-17 15:00:00,Ligue 1,2023-2024,16,Sun,Lorient,Strasbourg,W_Away,D,0.40,2023-12-17 15:00:00-Lorient-vs-Strasbourg
6,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Paris Saint Germain,Metz,W_Home,W_Home,0.72,2023-12-20 21:00:00-Metz-vs-Paris Saint Germain
7,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Nice,Lens,W_Home,W_Home,0.45,2023-12-20 21:00:00-Lens-vs-Nice
8,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Brest,Lorient,W_Home,W_Home,0.49,2023-12-20 21:00:00-Brest-vs-Lorient
9,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Reims,Le Havre,W_Home,D,0.44,2023-12-20 21:00:00-Le Havre-vs-Reims


In [7]:
selected_columns = ["Result", "DateTime"] + [col for col in resultat.columns if col.endswith(('Lag1_Home', 'Lag1_Away', 'Lag_Home', 'Lag_Away'))]
cutoff_date = datetime(2023, 12, 20, 21, 0, 0)
X = resultat[selected_columns].copy() 
X["Result"] = X['Result'].astype('category')

# Séparation des données
X_train = X[resultat['DateTime'] <= cutoff_date].dropna()
y_train = X_train["Result"]
X_train.drop(columns=['Result', 'DateTime'], errors='ignore', inplace=True)
X_test = X[resultat['DateTime'] > cutoff_date].drop(columns=['Result', 'DateTime'], errors='ignore')


In [9]:
X_test

,Points_Cum_Lag1_Home,GD_Cum_Lag1_Home,GF_Cum_Lag1_Home,GA_Cum_Lag1_Home,Classement_Lag1_Home,CumulativeWins_Lag1_Home,CumulativeDraws_Lag1_Home,CumulativeLosses_Lag1_Home,Moyenne_Total Shots_Lag_Home,Moyenne_Shots on Target_Lag_Home,Moyenne_Shots on Target %_Lag_Home,Moyenne_Goals per Shot_Lag_Home,Moyenne_Total Touches_Lag_Home,Moyenne_Touches in Defensive Penalty Area_Lag_Home,Moyenne_Touches in Defensive Third_Lag_Home,Moyenne_Touches in Midfield Third_Lag_Home,Moyenne_Touches in Attacking Third_Lag_Home,Moyenne_Touches in Attacking Penalty Area_Lag_Home,Moyenne_Dribbles Attempted_Lag_Home,Moyenne_Successful Dribbles_Lag_Home,Moyenne_Successful Dribble %_Lag_Home,Moyenne_Total Carries_Lag_Home,Moyenne_Total Carry Distance_Lag_Home,Moyenne_Progressive Carry Distance_Lag_Home,Moyenne_Progressive Carries_Lag_Home,Moyenne_Carries into Final Third_Lag_Home,Moyenne_Carries into Penalty Area_Lag_Home,Moyenne_Tackles_Lag_Home,Moyenne_Tackles Won_Lag_Home,Moyenne_Tackles in Defensive Third_Lag_Home,Moyenne_Tackles in Midfield Third_Lag_Home,Moyenne_Tackles in Attacking Third_Lag_Home,Moyenne_Dribblers Tackled_Lag_Home,Moyenne_Total Dribbles Against_Lag_Home,Moyenne_Defensive Dribblers Win %_Lag_Home,Moyenne_Interceptions_Lag_Home,Moyenne_Errors Leading to Goal_Lag_Home,Moyenne_Key Passes_Lag_Home,Moyenne_Passes Completed_Lag_Home,Moyenne_Passes Attempted_Lag_Home,Moyenne_Passes into Final Third_Lag_Home,Moyenne_Progressive Passes_Lag_Home,Moyenne_Shots on Target Against_Lag_Home,Moyenne_Keeper Saves_Lag_Home,Moyenne_Keeper Save Percentage_Lag_Home,Points_Cum_Lag1_Away,GD_Cum_Lag1_Away,GF_Cum_Lag1_Away,GA_Cum_Lag1_Away,Classement_Lag1_Away,CumulativeWins_Lag1_Away,CumulativeDraws_Lag1_Away,CumulativeLosses_Lag1_Away,Moyenne_Total Shots_Lag_Away,Moyenne_Shots on Target_Lag_Away,Moyenne_Shots on Target %_Lag_Away,Moyenne_Goals per Shot_Lag_Away,Moyenne_Total Touches_Lag_Away,Moyenne_Touches in Defensive Penalty Area_Lag_Away,Moyenne_Touches in Defensive Third_Lag_Away,Moyenne_Touches in Midfield Third_Lag_Away,Moyenne_Touches in Attacking Third_Lag_Away,Moyenne_Touches in Attacking Penalty Area_Lag_Away,Moyenne_Dribbles Attempted_Lag_Away,Moyenne_Successful Dribbles_Lag_Away,Moyenne_Successful Dribble %_Lag_Away,Moyenne_Total Carries_Lag_Away,Moyenne_Total Carry Distance_Lag_Away,Moyenne_Progressive Carry Distance_Lag_Away,Moyenne_Progressive Carries_Lag_Away,Moyenne_Carries into Final Third_Lag_Away,Moyenne_Carries into Penalty Area_Lag_Away,Moyenne_Tackles_Lag_Away,Moyenne_Tackles Won_Lag_Away,Moyenne_Tackles in Defensive Third_Lag_Away,Moyenne_Tackles in Midfield Third_Lag_Away,Moyenne_Tackles in Attacking Third_Lag_Away,Moyenne_Dribblers Tackled_Lag_Away,Moyenne_Total Dribbles Against_Lag_Away,Moyenne_Defensive Dribblers Win %_Lag_Away,Moyenne_Interceptions_Lag_Away,Moyenne_Errors Leading to Goal_Lag_Away,Moyenne_Key Passes_Lag_Away,Moyenne_Passes Completed_Lag_Away,Moyenne_Passes Attempted_Lag_Away,Moyenne_Passes into Final Third_Lag_Away,Moyenne_Progressive Passes_Lag_Away,Moyenne_Shots on Target Against_Lag_Away,Moyenne_Keeper Saves_Lag_Away,Moyenne_Keeper Save Percentage_Lag_Away


In [39]:
resultat[(resultat["Saison"] == "2023-2024") & (resultat["Round"] == 17)][["DateTime", "Comp", "Saison", "Round", "Day","Team Home", "Team Away", "Result",  "MatchID"]]

,DateTime,Comp,Saison,Round,Day,Team Home,Team Away,Result,MatchID
1806,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Brest,Lorient,W_Home,2023-12-20 21:00:00-Brest-vs-Lorient
1815,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Clermont Foot,Rennes,W_Away,2023-12-20 21:00:00-Clermont Foot-vs-Rennes
1857,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Lyon,Nantes,W_Home,2023-12-20 21:00:00-Lyon-vs-Nantes
1890,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Montpellier,Marseille,D,2023-12-20 21:00:00-Marseille-vs-Montpellier
1908,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Nice,Lens,W_Home,2023-12-20 21:00:00-Lens-vs-Nice
1917,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Paris Saint Germain,Metz,W_Home,2023-12-20 21:00:00-Metz-vs-Paris Saint Germain
1925,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Reims,Le Havre,W_Home,2023-12-20 21:00:00-Le Havre-vs-Reims
1943,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Strasbourg,Lille,W_Home,2023-12-20 21:00:00-Lille-vs-Strasbourg
1951,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Toulouse,Monaco,W_Away,2023-12-20 21:00:00-Monaco-vs-Toulouse


In [9]:
modelisation(resultat, ajd).to_csv("futur_result.csv")

/home/onyxia/work/Projet-python/Fbref_alex/TestAutom/fonctions_preprocess.py:442: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  y_train = X_train[df['DateTime'] <= cutoff_date]["Result"]


In [9]:
ajd = datetime.now()
A = pd.read_csv("/home/onyxia/work/Projet-python/Fbref_alex/futur_result.csv", index_col=0)
A["DateTime"] = pd.to_datetime(A["DateTime"])
derniere_date = A["DateTime"].max()

if derniere_date <= ajd:

    base_actuelle = pd.read_csv("/home/onyxia/work/Projet-python/Fbref_alex/SOCCER_241223.csv", index_col=0)
    data = scrape_latest_ligue1_data()
    base_supplementaire = data[0]
    base_updated = add_new_matches(base_initiale=base_actuelle, base_nouvelle=base_supplementaire)
    base_updated.to_csv("/home/onyxia/work/Projet-python/Fbref_alex/SOCCER_241223.csv")

    recup_matchweek = data[1].copy()
    recup_matchweek=find_futur_matchweeks(recup_matchweek, mapping_equipe)

    if recup_matchweek is None:
        print("Pas de nouveaux matchs à récupérer")
    else:
        
        recup_matchweek2 = recup_matchweek.copy()
        recup_matchweek2 = preprocess_initial(recup_matchweek2, mapping_equipe)
        recup_matchweek2 = renommer_colonnes(recup_matchweek2)
        recup_matchweek2 = columns_to_keep(recup_matchweek2)

        base_updated2 = base_updated.copy()
        base_updated2 = preprocess_initial(base_updated2, mapping_equipe)
        base_updated2 = renommer_colonnes(base_updated2)
        base_updated2 = columns_to_keep(base_updated2)
        base_updated2 = preprocess_variables(base_updated2)



        resultat = pd.concat([base_updated2, recup_matchweek2], sort=False).reset_index(drop= True)
        resultat = preparation_model(resultat)
        resultat = preprocess_data(resultat)

        # Fusionner les DataFrames
        A = A.merge(resultat[['MatchID', 'Result']], on='MatchID', how='left', suffixes=('', '_from_resultat'))
        A['Result'].update(A['Result_from_resultat'])
        A.drop(columns=['Result_from_resultat'], inplace=True)

        resultat = modelisation(resultat, ajd)
        A = pd.concat([A, resultat]).sort_values(by="DateTime", ascending=False).reset_index(drop= True)

        A.to_csv("futur_result.csv")
        

        


    #base_updated.to_csv("/home/onyxia/work/Projet-python/Fbref_alex/SOCCER_241223.csv")
    #base_updated.to_csv("Fbref_alex/SOCCER_241223.csv")
    


    #recup_matchweek.to_csv("/home/onyxia/work/Projet-python/Fbref_alex/recup_matchweek.csv")
    #recup_matchweek.to_csv("Fbref_alex/recup_matchweek.csv")
    

/home/onyxia/work/Projet-python/Fbref_alex/scrapping.py:92: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  team_data = pd.read_html(team_response.text, match="Scores")[0]
/home/onyxia/work/Projet-python/Fbref_alex/scrapping.py:158: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/home/onyxia/work/Projet-python/Fbref_alex/scrapping.py:158: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  detailed_stats = pd.read_html(stats_response.text)[0]
/home/onyxia/work/Projet-python/Fbref_alex/scrapping.py:158: FutureWarning: Passing literal html to 'read_html' is deprecated and wil

In [10]:
pd.read_csv("/home/onyxia/work/Projet-python/Fbref_alex/futur_result.csv",index_col=0)

,DateTime,Comp,Saison,Round,Day,Team Home,Team Away,Result,Predicted_Result,Prediction_Probability,MatchID
0,2024-01-14 20:45:00,Ligue 1,2023-2024,18,Sun,Lens,Paris Saint Germain,NaN,W_Home,0.39,2024-01-14 20:45:00-Lens-vs-Paris Saint Germain
1,2024-01-14 17:05:00,Ligue 1,2023-2024,18,Sun,Le Havre,Lyon,NaN,W_Away,0.48,2024-01-14 17:05:00-Le Havre-vs-Lyon
2,2024-01-14 15:00:00,Ligue 1,2023-2024,18,Sun,Nantes,Clermont Foot,NaN,W_Home,0.43,2024-01-14 15:00:00-Clermont Foot-vs-Nantes
3,2024-01-14 15:00:00,Ligue 1,2023-2024,18,Sun,Metz,Toulouse,NaN,W_Away,0.43,2024-01-14 15:00:00-Metz-vs-Toulouse
4,2024-01-14 15:00:00,Ligue 1,2023-2024,18,Sun,Brest,Montpellier,NaN,W_Home,0.56,2024-01-14 15:00:00-Brest-vs-Montpellier
5,2024-01-14 13:00:00,Ligue 1,2023-2024,18,Sun,Lille,Lorient,NaN,W_Home,0.62,2024-01-14 13:00:00-Lille-vs-Lorient
6,2024-01-13 21:00:00,Ligue 1,2023-2024,18,Sat,Rennes,Nice,NaN,W_Away,0.37,2024-01-13 21:00:00-Nice-vs-Rennes
7,2024-01-13 17:00:00,Ligue 1,2023-2024,18,Sat,Monaco,Reims,NaN,W_Home,0.62,2024-01-13 17:00:00-Monaco-vs-Reims
8,2024-01-12 21:00:00,Ligue 1,2023-2024,18,Fri,Marseille,Strasbourg,NaN,W_Home,0.44,2024-01-12 21:00:00-Marseille-vs-Strasbourg
9,2023-12-20 21:00:00,Ligue 1,2023-2024,17,Wed,Toulouse,Monaco,W_Away,W_Away,0.55,2023-12-20 21:00:00-Monaco-vs-Toulouse


In [100]:
recup_matchweek = data[1].copy()

In [101]:
recup_matchweek

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team
0,2023-08-12,21:00,Ligue 1,Matchweek 1,Sat,Home,D,0.0,0.0,Lorient,1.2,0.1,78.0,47000.0,Danilo Pereira,4-3-3,Bastien Dechepy,Match Report,NaN,Paris Saint Germain
1,2023-08-19,21:00,Ligue 1,Matchweek 2,Sat,Away,D,1.0,1.0,Toulouse,2.1,1.2,76.0,27332.0,Marquinhos,4-3-3,Thomas Léonard,Match Report,NaN,Paris Saint Germain
2,2023-08-26,21:00,Ligue 1,Matchweek 3,Sat,Home,W,3.0,1.0,Lens,1.5,0.5,66.0,47864.0,Marquinhos,4-4-2,François Letexier,Match Report,NaN,Paris Saint Germain
3,2023-09-03,20:45,Ligue 1,Matchweek 4,Sun,Away,W,4.0,1.0,Lyon,4.5,2.1,69.0,51657.0,Marquinhos,4-4-2,Eric Wattellier,Match Report,NaN,Paris Saint Germain
4,2023-09-15,21:00,Ligue 1,Matchweek 5,Fri,Home,L,2.0,3.0,Nice,1.0,1.3,69.0,47874.0,Kylian Mbappé,4-3-3,Clément Turpin,Match Report,NaN,Paris Saint Germain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678,2024-04-21,NaN,Ligue 1,Matchweek 30,Sun,Away,NaN,NaN,NaN,Lens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Clermont Foot
679,2024-04-28,NaN,Ligue 1,Matchweek 31,Sun,Home,NaN,NaN,NaN,Reims,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Clermont Foot
680,2024-05-03,NaN,Ligue 1,Matchweek 32,Fri,Away,NaN,NaN,NaN,Monaco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Clermont Foot
681,2024-05-11,21:00,Ligue 1,Matchweek 33,Sat,Home,NaN,NaN,NaN,Lyon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Clermont Foot


In [102]:
def find_futur_matchweeks2(df, mapping_equipe):

    # Supprimer les lignes où les colonnes 'Date', 'Time' et 'Round' sont manquantes.
    df.dropna(subset=["Date", "Time", "Round"], inplace=True)

    # Prétraiter le DataFrame en utilisant la fonction 'preprocess_intitial' et le mapping des équipes.
    df = preprocess_initial(df, mapping_equipe)

    # Obtenir la date et l'heure actuelles.
    ajd = datetime.now()

    # Filtrer pour garder seulement les matchs programmés après la date et l'heure actuelles.
    df = df[df['DateTime'] >= ajd]

    # Trier le DataFrame en fonction de la colonne 'DateTime' dans l'ordre croissant.
    df = df.sort_values(by='DateTime')

    # Si le DataFrame n'est pas vide, obtenir la date du premier match à venir.
    # Sinon, définir 'premiere_date_proche' à None.
    if df.empty != True:
        premiere_date_proche = df['DateTime'].iloc[0]
    else:
        return None
    
    # Calculer la date qui est 10 jours après la 'premiere_date_proche'.
    dix_jours = timedelta(days=10) + premiere_date_proche

    # Filtrer pour garder seulement les matchs programmés dans les 10 jours suivant la 'premiere_date_proche'.
    df = df[df['DateTime'] <= dix_jours]

    # Si 'premiere_date_proche' est None, retourner None.
    if df.empty == True:
        return None
    else:# Retourner le DataFrame s'il n'est pas vide, sinon retourner None.
        return df





In [103]:
recup_matchweek = find_futur_matchweeks2(recup_matchweek, mapping_equipe)

In [104]:
recup_matchweek

,DateTime,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes,Team,Saison,MatchID
218,2024-01-12 21:00:00,Ligue 1,18,Fri,Home,NaN,NaN,NaN,Strasbourg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Marseille,2023-2024,Marseille_Strasbourg
334,2024-01-12 21:00:00,Ligue 1,18,Fri,Away,NaN,NaN,NaN,Marseille,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Strasbourg,2023-2024,Strasbourg_Marseille
97,2024-01-13 17:00:00,Ligue 1,18,Sat,Home,NaN,NaN,NaN,Reims,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Monaco,2023-2024,Monaco_Reims
299,2024-01-13 17:00:00,Ligue 1,18,Sat,Away,NaN,NaN,NaN,Monaco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Reims,2023-2024,Reims_Monaco
375,2024-01-13 21:00:00,Ligue 1,18,Sat,Home,NaN,NaN,NaN,Nice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Rennes,2023-2024,Rennes_Nice
62,2024-01-13 21:00:00,Ligue 1,18,Sat,Away,NaN,NaN,NaN,Rennes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Nice,2023-2024,Nice_Rennes
631,2024-01-14 13:00:00,Ligue 1,18,Sun,Away,NaN,NaN,NaN,Lille,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Lorient,2023-2024,Lorient_Lille
175,2024-01-14 13:00:00,Ligue 1,18,Sun,Home,NaN,NaN,NaN,Lorient,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Lille,2023-2024,Lille_Lorient
482,2024-01-14 15:00:00,Ligue 1,18,Sun,Home,NaN,NaN,NaN,Clermont Foot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Nantes,2023-2024,Nantes_Clermont Foot
666,2024-01-14 15:00:00,Ligue 1,18,Sun,Away,NaN,NaN,NaN,Nantes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Head-to-Head,NaN,Clermont Foot,2023-2024,Clermont Foot_Nantes


In [84]:
A = pd.read_csv("/home/onyxia/work/Projet-python/Fbref_alex/futur_result.csv", index_col=0)
A["DateTime"] = pd.to_datetime(A["DateTime"])
derniere_date = A["DateTime"].max()

recup_matchweek2 = recup_matchweek.copy()
recup_matchweek2 = preprocess_initial(recup_matchweek2, mapping_equipe)
recup_matchweek2 = renommer_colonnes(recup_matchweek2)
recup_matchweek2 = columns_to_keep(recup_matchweek2)

base_updated2 = base_updated.copy()
base_updated2 = preprocess_initial(base_updated2, mapping_equipe)
base_updated2 = renommer_colonnes(base_updated2)
base_updated2 = columns_to_keep(base_updated2)
base_updated2 = preprocess_variables(base_updated2)



In [85]:
resultat2 = pd.concat([base_updated2, recup_matchweek2], sort=False).reset_index(drop= True)
resultat2 = preparation_model(resultat2)
resultat2 = preprocess_data(resultat2)


In [86]:

# Fusionner les DataFrames
A = A.merge(resultat2[['MatchID', 'Result']], on='MatchID', how='left', suffixes=('', '_from_resultat2'))
A['Result'].update(A['Result_from_resultat2'])
A.drop(columns=['Result_from_resultat2'], inplace=True)


/tmp/ipykernel_335950/1382058102.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['D' 'W_Home' 'W_Away' 'W_Away' 'D' 'W_Away' 'W_Home' 'W_Home' 'W_Home'
 'W_Home' 'W_Home' 'D' 'W_Home' 'W_Away' 'W_Away']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  A['Result'].update(A['Result_from_resultat2'])


In [87]:
resultat2 = modelisation(resultat2, ajd)


In [93]:
A = pd.concat([A, resultat2]).sort_values(by="DateTime", ascending=False).reset_index(drop= True)


In [94]:
A

,DateTime,Comp,Saison,Round,Day,Team Home,Team Away,Result,Predicted_Result,Prediction_Probability,MatchID
0,2024-01-14 20:45:00,Ligue 1,2023-2024,18,Sun,Lens,Paris Saint Germain,NaN,W_Home,0.39,2024-01-14 20:45:00-Lens-vs-Paris Saint Germain
1,2024-01-14 20:45:00,Ligue 1,2023-2024,18,Sun,Lens,Paris Saint Germain,NaN,W_Home,0.39,2024-01-14 20:45:00-Lens-vs-Paris Saint Germain
2,2024-01-14 17:05:00,Ligue 1,2023-2024,18,Sun,Le Havre,Lyon,NaN,W_Away,0.48,2024-01-14 17:05:00-Le Havre-vs-Lyon
3,2024-01-14 17:05:00,Ligue 1,2023-2024,18,Sun,Le Havre,Lyon,NaN,W_Away,0.48,2024-01-14 17:05:00-Le Havre-vs-Lyon
4,2024-01-14 15:00:00,Ligue 1,2023-2024,18,Sun,Nantes,Clermont Foot,NaN,W_Home,0.43,2024-01-14 15:00:00-Clermont Foot-vs-Nantes
5,2024-01-14 15:00:00,Ligue 1,2023-2024,18,Sun,Metz,Toulouse,NaN,W_Away,0.43,2024-01-14 15:00:00-Metz-vs-Toulouse
6,2024-01-14 15:00:00,Ligue 1,2023-2024,18,Sun,Brest,Montpellier,NaN,W_Home,0.56,2024-01-14 15:00:00-Brest-vs-Montpellier
7,2024-01-14 15:00:00,Ligue 1,2023-2024,18,Sun,Metz,Toulouse,NaN,W_Away,0.43,2024-01-14 15:00:00-Metz-vs-Toulouse
8,2024-01-14 15:00:00,Ligue 1,2023-2024,18,Sun,Brest,Montpellier,NaN,W_Home,0.56,2024-01-14 15:00:00-Brest-vs-Montpellier
9,2024-01-14 15:00:00,Ligue 1,2023-2024,18,Sun,Nantes,Clermont Foot,NaN,W_Home,0.43,2024-01-14 15:00:00-Clermont Foot-vs-Nantes
